In [32]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [33]:
import numpy as np
import pandas as pd

In [34]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [35]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import join, outerjoin, MetaData
from sqlalchemy import Table, Column, Integer, String, Float

In [36]:
# Connect to database
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
meta = MetaData()
# Create an inspector and connect it to the engine
inspector = inspect(engine)
# dir(inspector)
# View table names
tables = inspector.get_table_names()
tables

['measurement', 'station']

In [37]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
# reflect the tables
tables = Base.classes.keys()

In [38]:
# We can view all of the classes that automap found
tables

['measurement', 'station']

In [39]:
# Save references to each table
Measurement = Base.classes['measurement']
Station = Base.classes['station']
# dir(Station)
# dir(Measurement)

In [65]:
# Create our session (link) from Python to the DB
session = Session(engine)
session.query(Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).first()

('USC00519397', '2010-01-01', 0.08, 65.0)

In [66]:
session.query(Station.station, Station.name).all()

[('USC00519397', 'WAIKIKI 717.2, HI US'),
 ('USC00513117', 'KANEOHE 838.1, HI US'),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US'),
 ('USC00517948', 'PEARL CITY, HI US'),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US'),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US'),
 ('USC00519281', 'WAIHEE 837.5, HI US'),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US'),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US')]

# Exploratory Climate Analysis

In [67]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Query only Honolulu Station to find end date
hawaii = session.query(Measurement.date, Measurement.prcp)
hawaii.first()
count = hawaii.count()

# find end date based on count - 1 above
hawaii[count-1]

# Calculate the date 1 year ago from the last data point in the database

# a_date = datetime.date(2015, 10, 10) 
# days = datetime.timedelta(5)
# new_date = a_date - days

end_date = dt.date(2017, 8, 23)
year_days = dt.timedelta(365)
begin_date = end_date - year_days

# Perform a query to retrieve the data and precipitation scores

hawaii_data_year = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date>=begin_date)

# Save the query results as a Pandas DataFrame and set the index to the date column

hawaii_data_year_df = pd.read_sql(hawaii_data_year.statement, conn)

hawaii_data_year_df = hawaii_data_year_df.set_index('date')

# Sort the dataframe by date

hawaii_data_year_df = hawaii_data_year_df.sort_values(by="date")

hawaii_data_year_df.head(100)

# Use Pandas Plotting with Matplotlib to plot the data

#df = pd.DataFrame({'lab':['A', 'B', 'C'], 'val':[10, 30, 20]})
#ax = df.plot.bar(x='lab', y='val', rot=0)
#ax = hawaii_data_year_df.plot.bar(y='prcp', rot=0)

#hist = df.hist(bins=40)
#hist = hawaii_data_year_df.hist(bins=8)

#hawaii_data_year_df.hist(column='prcp')

prcp
date            
2016-08-23  0.00
2016-08-23   NaN
2016-08-23  1.79
2016-08-23  0.05
2016-08-23  0.15
...          ...
2016-09-07  0.05
2016-09-07  0.11
2016-09-07  0.10
2016-09-07  1.35
2016-09-07  0.23

[100 rows x 1 columns]

In [19]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
